##  Malaria Diagnosis
-  Task
- Data    (From Tensorflow datasets)
- Modelling (CNN)
- Error Sanctioning
- Training and Optimization
- Performance   Measurement
- Validation and testing
- Corrective measures

### Task
 we have a segment of cell we have to tell whether it is parasitized or unparasitized

## Dataset Preparation

In [1]:
### Importing libraries
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds
from keras.src.ops import dtype
from pandas.io.sas.sas_constants import dataset_length

In [2]:
ds,ds_info=tfds.load('malaria',with_info=True,as_supervised=True, shuffle_files=True,split=['train'])
print(ds)
print(ds_info)

E:\AI 2025\CV\TensorFlow_for_CV\myenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Dl Completed...: 0 url [00:00, ? url/s]
Dl Size...: 0 MiB [00:00, ? MiB/s]

Dl Completed...:   0%|          | 0/1 [00:00<?, ? url/s]
Dl Size...: 0 MiB [00:00, ? MiB/s]

Dl Completed...: 100%|██████████| 1/1 [00:00<00:00, 70.30 url/s]
Dl Size...: 0 MiB [00:00, ? MiB/s]

Dl Size...:   0%|          | 0/353452851 [00:00<?, ? MiB/s]

Dl Size...: 100%|██████████| 353452851/353452851 [00:00<00:00, 15233604681.20 MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]
Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]
Generating train examples...: 0 examples [00:00, ? examples/s]
Generating train examples...: 2673 examples [00:01, 2672.29 examples/s]
Generating train examples...: 5724 examples [00:02, 2892.31 examples/s]
Generating train examples...: 8617 examples [00:03, 2754.56 examples/s]
Generating train examples...: 11379 examples [00:04, 2400.52 examples/s]
Generating train examples...: 14114 examples [00:05, 2500.16 examples/s]
Generating train examples...: 16804 example

Dataset malaria downloaded and prepared to C:\Users\wajah\tensorflow_datasets\malaria\1.0.0. Subsequent calls will reuse this data.
[<_PrefetchDataset element_spec=(TensorSpec(shape=(None, None, 3), dtype=tf.uint8, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>]
tfds.core.DatasetInfo(
    name='malaria',
    full_name='malaria/1.0.0',
    description="""
    The Malaria dataset contains a total of 27,558 cell images with equal instances
    of parasitized and uninfected cells from the thin blood smear slide images of
    segmented cells.
    """,
    homepage='https://lhncbc.nlm.nih.gov/publication/pub9932',
    data_dir='C:\\Users\\wajah\\tensorflow_datasets\\malaria\\1.0.0',
    file_format=tfrecord,
    download_size=Unknown size,
    dataset_size=317.62 MiB,
    features=FeaturesDict({
        'image': Image(shape=(None, None, 3), dtype=uint8),
        'label': ClassLabel(shape=(), dtype=int64, num_classes=2),
    }),
    supervised_keys=('image', 'label'),
    disab

In [3]:
for data in ds[0].take(1):
    print(data)

(<tf.Tensor: shape=(151, 115, 3), dtype=uint8, numpy=
array([[[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       ...,

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]]], dtype=uint8)>, <tf.Tensor: shape=(), dtype=int64, numpy=1>)


In [14]:
Train_Ratio=0.8
Val_Ratio=0.1
Test_Ratio=0.1
dataset=tf.data.Dataset.range(10)
print(list(dataset.as_numpy_iterator()))
dataset_length=len(dataset)

[np.int64(0), np.int64(1), np.int64(2), np.int64(3), np.int64(4), np.int64(5), np.int64(6), np.int64(7), np.int64(8), np.int64(9)]


In [15]:
train_dataset=dataset.take(int(Train_Ratio*dataset_length))
# print(train_dataset)
print(list(train_dataset.as_numpy_iterator()))

[np.int64(0), np.int64(1), np.int64(2), np.int64(3), np.int64(4), np.int64(5), np.int64(6), np.int64(7)]


In [26]:
val_test_dataset=dataset.skip(int(Train_Ratio*dataset_length))
print(list(val_test_dataset.as_numpy_iterator()))
val_dataset=val_test_dataset.take(int(Val_Ratio*dataset_length))
print(list(val_dataset.as_numpy_iterator()))

[np.int64(8), np.int64(9)]
[np.int64(8)]


In [27]:
test_dataset=val_test_dataset.skip(int(Val_Ratio*dataset_length))
print(list(test_dataset.as_numpy_iterator()))

[np.int64(9)]
